# This notebook walks through the basics of using TASM
It covers
- Loading metadata
- Loading videos and storing them with and without tiles
- Retrieving objects
- (coming) Retrieving full frames
- Automatically tiling videos as queries execute

## Setup: loading a video and detections.

In [ ]:
# Remove previously-stored videos, data.
import os
import shutil

if os.path.exists('resources'):
    shutil.rmtree('resources')
if os.path.exists('labels.db'):
    os.remove('labels.db')

In [ ]:
VIDEO_PATH = 'data/birds.mp4'
DETECTIONS_PATH = 'data/birds.pkl'

### Load detections.

In [ ]:
import pandas as pd

detections = pd.read_pickle(DETECTIONS_PATH)
detections = detections.astype({'x1': int, 'y1': int, 'x2': int, 'y2': int})
detections.head()

### View detections on a few frames.
Is it the most exciting video? No. But at one point I thought it was exciting enough to capture, so here we are.

In [ ]:
import cv2
import matplotlib.pyplot as plt

def draw_box(img, label, x1, y1, x2, y2):
    pt1 = (x1, y1)
    pt2 = (x2, y2)
    cv2.rectangle(img, pt1, pt2, (255, 0, 0), 2)
    cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 2, [255, 0, 0], 2)
    return img

num_frames = 5
vc = cv2.VideoCapture(VIDEO_PATH)

f, axs = plt.subplots(1, num_frames, figsize=(15, 15))
[ax.axis('off') for ax in axs.ravel()]
for f in range(num_frames):
    ret, frame = vc.read()
    if not ret:
        break
        
    objects = detections[detections.frame == f]
    for i, row in objects.iterrows():
        frame = draw_box(frame, row.label, row.x1, row.y1, row.x2, row.y2)

    axs[f].imshow(frame)

## Loading the video and detections into TASM.

In [ ]:
import os
import tasm

t = tasm.TASM()

# First, load the video without tiling into TASM.
untiled_video_name = 'birds-untiled'
t.store(VIDEO_PATH, untiled_video_name)

# We can also store it with a uniform layout, e.g. 2x2.
rows = 2
cols = 2
t.store_with_uniform_layout(VIDEO_PATH, 'birds-2x2', rows, cols)

In [ ]:
# Add metadata about the video. 
# We could use t.add_metadata(video, label, frame, x1, y1, x2, y2).
# However, since we have all of the metadata available, we can use t.add_bulk_metadata to add all of it at once.
metadata_id = 'birds'
label = 'bird'
metadata_info = [
    tasm.MetadataInfo(metadata_id, r.label, r.frame, r.x1, r.y1, r.x2, r.y2) 
    for _, r in detections.iterrows()
]
t.add_bulk_metadata(metadata_info)

In [ ]:
# Now, we can use the metadata to tile around birds.
tiled_video_name = 'birds-birds'
t.store_with_nonuniform_layout(VIDEO_PATH, tiled_video_name, metadata_id, label)

## Retrieving objects of interest using TASM.

In [ ]:
# First, let's retrieve a few birds from the un-tiled video.
first_frame_inclusive = 0
last_frame_exclusive = 5
tiled_selection = t.select(tiled_video_name, metadata_id, label, first_frame_inclusive, last_frame_exclusive)

# Inspect the returned instances.
num_birds = 0
num_cols = 5
f, axs = plt.subplots(1, num_cols)
[ax.axis('off') for ax in axs.ravel()]
while True:
    bird = tiled_selection.next()
    if bird.is_empty():
        break
    w = bird.width()
    h = bird.height()
    axs[num_birds].imshow(bird.numpy_array())
    num_birds += 1
print(f'Detected {num_birds} birds.')

In [ ]:
import time

def time_selection(video_name, metadata_id, label, first_frame=None, last_frame=None):
    start = time.perf_counter()
    selection = t.select(video_name, metadata_id, label, first_frame, last_frame) \
                        if first_frame is not None and last_frame is not None \
                        else t.select(video_name, metadata_id, label)
    num_objs = 0
    while True:
        obj = selection.next()
        if obj.is_empty():
            break
        num_objs += 1
    end = time.perf_counter()
    return end - start, num_objs

In [ ]:
# Compare the time to retrieve all of the birds from the untiled video from the tiled video.
# By not specifying first frame and last frame, TASM selects all instances of the object from the video.
dur, num_birds = time_selection(untiled_video_name, metadata_id, label)
print(f'Retrieved {num_birds} birds in {dur}')

dur, num_birds = time_selection(tiled_video_name, metadata_id, label)
print(f'Retrieved {num_birds} birds in {dur}')

## <span style="color:blue">Coming: retrieving full frames from a tiled video</span>

## Automatically tiling a video.

In [ ]:
# Start with an untiled video.
t = tasm.TASM()
incrementally_tiled_name = 'birds-incremental'
t.store(VIDEO_PATH, incrementally_tiled_name)

In [ ]:
# Perform 10 selections on the un-tiled video over the first 5 seconds.
first_frame_inclusive = 0
last_frame_exclusive = 150
num_selections = 10
start = time.perf_counter()
for i in range(num_selections):
    time_selection(incrementally_tiled_name, metadata_id, label, first_frame_inclusive, last_frame_exclusive)
print(f'{num_selections} selections over the untiled video took {time.perf_counter() - start}')

In [ ]:
# To automatically tile a video, we must first activate it so TASM will start tracking regret.
# Activate regret-based re-tiling.
t.activate_regret_based_tiling(incrementally_tiled_name, metadata_id)

# Perform 0 selections with incremental tiling over the first 5 seconds.
# Currently, the call to re-tile must be manually specified.
# Moving this into the selection calls is tracked at https://github.com/maureendaum/TASM/issues/5.
start = time.perf_counter()
for i in range(num_selections):
    time_selection(incrementally_tiled_name, metadata_id, label, first_frame_inclusive, last_frame_exclusive)
    t.retile_based_on_regret(incrementally_tiled_name)
print(f'{num_selections} selections with incremental tiling took {time.perf_counter() - start}')